# Loading data to Supabase Posgres Database

We have decided to try and host the database using Supabase as the backend. Hosting this on a Supabase's cloud, we could more easily scale and build applications that can access that database. We will eventually try this with the isochrone project.

In [1]:
import pandas as pd
import os
from supabase import create_client, Client

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Supabase API

We have created a database on Supabase off-code. This was created using my(nabilersyad) account under that organization. The database mirrors the SQLite database.
Details on the created database below.

SUPABASE_URL = 'saved locally in the shell. Get the details from supabase dashboard'  
SUPABASE_KEY = 'saved in the shell. Get the details from supabase dashboard'

database: transit-stations-database  
tables: 1. stations  
        2. entrances  
        3. station_entrances  

In [2]:
#declaring the supabase client we will working with
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [3]:
#signin a user
#sign_in_response = supabase.auth.sign_in_with_password({"email": "nabile95@hotmail.com", "password": ""})


In [4]:

#user = supabase.auth.get_user()


## Stations table

Combining the various CSVs of different cities and storing them in one table in the database

In [5]:

# Define the directory of cleansed data to transfer to sql
cleansed_data_directory = 'data_cleansed'
cleansed_kl_file = 'klang_valley_stations_cleansed.csv'
cleansed_montreal_file = 'montreal_stations_cleansed.csv'
cleansed_singapore_file = 'singapore_stations_cleansed.csv'
cleansed_combined_file = 'combined_stations_cleansed.csv'

# read cleaned dataframes
kl_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_kl_file))
montreal_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_montreal_file))
singapore_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_singapore_file))


# Combine all the dataframes
stations_data_local = pd.concat([kl_data, montreal_data, singapore_data], axis=0, ignore_index=True)

stations_data_local.index.name = 'station_id'
stations_data_local.to_csv(os.path.join(cleansed_data_directory, cleansed_combined_file), index=True)


# Print out the combined dataframe
stations_data_local


,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
0,KL Sentral,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Outram Park MRT Station,TE17,NaN,1.280400,103.840100,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
532,Maxwell MRT Station,TE18,NaN,1.280600,103.844000,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN
533,Shenton Way MRT Station,TE19,NaN,1.277540,103.850770,TEL,Thomson-East Coast Line,NaN,Brown,#734538,Singapore,NaN,NaN,NaN


In [6]:
response = supabase.table('stations').select("*").execute()
data,_ = response
stations_data_supa = pd.DataFrame(data[1])
stations_data_supa.set_index('station_id',inplace=True)
stations_data_supa

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened
station_id,,,,,,,,,,,,,,
1,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
2,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
3,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
4,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
5,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Maxwell MRT Station,TE18,None,1.280600,103.844000,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
533,Shenton Way MRT Station,TE19,None,1.277540,103.850770,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None
534,Marina Bay MRT Station,TE20,None,1.275290,103.854810,TEL,Thomson-East Coast Line,None,Brown,#734538,Singapore,None,None,None


In [7]:
# Perform an outer join on the dataframes
merged_data = pd.merge(stations_data_local, stations_data_supa, how='outer', indicator=True)


In [8]:
# Case 1 and 2: rows exist in both dataframes
both_data = merged_data[merged_data['_merge'] == 'both']
both_data = both_data.drop(columns=['_merge'])


In [9]:
#both_data[stations_data_local.columns].equals(both_data[stations_data_supa.columns])

In [10]:
identical_rows_mask = [both_data.loc[i, stations_data_local.columns].equals(both_data.loc[i, stations_data_supa.columns])
                        for i in both_data.index]
identical_rows = both_data[identical_rows_mask]


In [11]:
different_rows_mask= [not b for b in identical_rows_mask]
different_rows = both_data[different_rows_mask]

# Replace all NaN values in the DataFrame with None
different_rows = different_rows.where(pd.notna(different_rows), None)

different_rows

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened


In [12]:

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('stations').update(row.to_dict()).eq('station_id', row['station_id']).execute()

# Case 3: rows in data_local but not in data_supabase
only_local = merged_data[merged_data['_merge'] == 'left_only']

# For these rows, you want to insert into data_supabase
data, error = supabase.table('stations').insert(only_local.to_dict('records')).execute()

# Case 4: rows not in data_local but in data_supabase
only_supabase = merged_data[merged_data['_merge'] == 'right_only']

# For these rows, you want to delete from data_supabase
for index, row in only_supabase.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('stations').delete().eq('station_id', row['station_id']).execute()


## Entrances tables

There are two entrances tables that we need to create. We'll call them entrances and station_entrances

In [20]:
cleansed_entrances = 'klang_valley_entrances_cleansed.csv'

# read cleaned entrances dataframes
entrances_data_local = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_entrances))

entrances_data_local = entrances_data_local.where(pd.notna(entrances_data_local), None)
entrances_data_local

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,1544031348,101.711374,3.145929,None,B
1,1631412559,101.604952,3.113222,None,None
2,2278515570,101.644077,3.050650,None,None
3,2686635178,101.699182,3.138565,None,C
4,3308608988,101.712717,3.158762,None,None
...,...,...,...,...,...
229,11052165924,101.694160,3.161349,None,None
230,11061429683,101.671755,3.214623,None,None
231,11061429685,101.672075,3.214698,None,None
232,11061429686,101.672324,3.214263,None,None


In [21]:
response = supabase.table('entrances').select("*").execute()
data,_ = response
entrances_data_supa = pd.DataFrame(data[1])
entrances_data_supa.set_index('entrance_id',inplace=True)
entrances_data_supa

,longitude,latitude,entrance_destination,entrance_name
entrance_id,,,,
4952299496,101.729057,3.111715,None,B
5261535560,101.607848,3.155939,None,C
5261535561,101.608202,3.155933,None,B
5261535570,101.609118,3.154898,None,A
5261535576,101.609131,3.155588,None,D
...,...,...,...,...
10949038884,101.681439,3.237779,None,None
9849592637,101.592337,3.200010,None,None
9849815460,101.639674,3.214082,None,B


In [29]:
merged_data

,entrance_id,longitude,latitude,entrance_destination,entrance_name,_merge
0,1.544031e+09,101.711374,3.145929,None,B,both
1,1.631413e+09,101.604952,3.113222,None,None,both
2,2.278516e+09,101.644077,3.050650,None,None,both
3,2.686635e+09,101.699182,3.138565,None,C,both
4,3.308609e+09,101.712717,3.158762,None,None,both
...,...,...,...,...,...,...
267,NaN,101.704748,3.041676,Jalan Utama,A,right_only
268,NaN,101.696541,3.149150,None,C,right_only
269,NaN,101.790085,2.982981,None,A,right_only
270,NaN,101.718154,3.162546,None,A,right_only


In [24]:
# Perform an outer join on the dataframes
merged_data = pd.merge(entrances_data_local, entrances_data_supa, how='outer', indicator=True)

# Case 1 and 2: rows exist in both dataframes
both_data = merged_data[merged_data['_merge'] == 'both']
both_data = both_data.drop(columns=['_merge'])

identical_rows_mask = [both_data.loc[i, entrances_data_local.columns].equals(both_data.loc[i, entrances_data_supa.columns])
                        for i in both_data.index]
identical_rows = both_data[identical_rows_mask]

different_rows_mask= [not b for b in identical_rows_mask]
different_rows = both_data[different_rows_mask]

# Replace all NaN values in the DataFrame with None
different_rows = different_rows.where(pd.notna(different_rows), None)

# Convert float to int for 'entrance_id'
different_rows['entrance_id'] = different_rows['entrance_id'].astype(int)


different_rows

,entrance_id,longitude,latitude,entrance_destination,entrance_name
0,1544031348,101.711374,3.145929,None,B
1,1631412559,101.604952,3.113222,None,None
2,2278515570,101.644077,3.050650,None,None
3,2686635178,101.699182,3.138565,None,C
4,3308608988,101.712717,3.158762,None,None
...,...,...,...,...,...
215,10830050822,101.687762,3.132827,None,C
216,10839997852,101.657207,2.949712,Off Persiaran APEC,A
218,10864116957,101.731475,3.165190,None,None
220,10949038882,101.680826,3.237606,None,None


In [25]:

# For the different rows, you want to update data_supabase with data from data_local
for index, row in different_rows.iterrows():
    # Use Supabase update method
    # Note: replace 'id' and 'your_table' with your actual id column name and table name
    data, error = supabase.table('entrances').update(row.to_dict()).eq('entrance_id', row['entrance_id']).execute()

# Case 3: rows in data_local but not in data_supabase
only_local = merged_data[merged_data['_merge'] == 'left_only']
only_local = only_local.drop(columns=['_merge'])
only_local['entrance_id'] = only_local['entrance_id'].astype(int)
only_local = only_local.where(pd.notna(only_local), None)

# For these rows, you want to insert into data_supabase
data, error = supabase.table('entrances').insert(only_local.to_dict('records')).execute()

# Case 4: rows not in data_local but in data_supabase
only_supabase = merged_data[merged_data['_merge'] == 'right_only']
only_supabase = only_supabase.drop(columns=['_merge'])
only_supabase['entrance_id'] = only_supabase['entrance_id'].astype(int)
only_supabase = only_supabase.where(pd.notna(only_supabase), None)


# For these rows, you want to delete from data_supabase
for index, row in only_supabase.iterrows():
    # Use Supabase delete method
    data, error = supabase.table('entrances').delete().eq('entrance_id', row['entrance_id']).execute()

ConnectTimeout: timed out

## Station Entrances tables

This table tracks the relationship between station and entrances

In [ ]:
cleansed_station_entrances= 'klang_valley_stations_entrances_relation_cleansed.csv'

station_entrances_data = pd.read_csv(os.path.join(cleansed_data_directory, cleansed_station_entrances))
